In [1]:
import pandas as pd
import torch
import numpy as np
from model import MFP,DataSet
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from torch.utils.data.dataloader import DataLoader
from torchinfo import summary
from tqdm.auto import tqdm
from torch.nn.utils import clip_grad_norm_

In [2]:
file_path = './updated_datasets_distrubutions/final_ds_knn.csv'
file_path_y = './updated_datasets_distrubutions/final_ds_knn_y.csv'

In [3]:
df = pd.read_csv(file_path)
df_y = pd.read_csv(file_path_y)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.to_numpy(), df_y.to_numpy(), test_size=0.20, random_state=42)

In [5]:
train_dataset = DataSet(X_train,y_train)
test_dataset = DataSet(X_test,y_test)
train_dataloader = DataLoader(train_dataset,batch_size=128,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=128)

In [6]:
model = MFP().to('cuda')
summary(model)

Layer (type:depth-idx)                   Param #
MFP                                      --
├─LeakyReLU: 1-1                         --
├─ReLU: 1-2                              --
├─Sigmoid: 1-3                           --
├─Tanh: 1-4                              --
├─Mish: 1-5                              --
├─SiLU: 1-6                              --
├─SELU: 1-7                              --
├─ELU: 1-8                               --
├─GELU: 1-9                              --
├─Softplus: 1-10                         --
├─Softsign: 1-11                         --
├─Softshrink: 1-12                       --
├─Linear: 1-13                           81,510
├─Linear: 1-14                           81,510
├─Linear: 1-15                           81,510
├─Linear: 1-16                           81,510
├─Linear: 1-17                           81,510
├─Linear: 1-18                           81,510
├─Linear: 1-19                           81,510
├─Linear: 1-20                           81

In [7]:
loss_fn = torch.nn.HuberLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4,weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer=optimizer,total_iters=1000,start_factor=1,end_factor=1e-1)
writer = SummaryWriter('./runs')

In [8]:
for i in tqdm(range(1000)):
    model.train()
    for (x,y) in train_dataloader:
        output = model(x)
        loss = loss_fn(output,y)
        loss.backward()
        clip_grad_norm_(model.parameters(),0.1)
        optimizer.step()
        optimizer.zero_grad()
    scheduler.step()
    model.eval()
    with torch.inference_mode():
        loss_values = []
        for (x,y) in (test_dataloader):
            output = model(x)
            loss = torch.mean(torch.abs(output-y))
            loss_values.append(loss.cpu().item())
        writer.add_scalar('loss_knn',np.mean(loss_values),i)

  0%|          | 0/1000 [00:00<?, ?it/s]